<div style="width:100%; height:140px">
    <img src="https://www.kuleuven.be/internationaal/thinktank/fotos-en-logos/ku-leuven-logo.png/image_preview" width = 300px, heigh = auto align=left>
</div>


KUL H02A5a Computer Vision: Group Assignment 2
---------------------------------------------------------------
Student numbers: <span style="color:blue">r0703889, r0909802, r0716758, r0916443, r0822692</span>. 

In this group assignment your team will delve into some deep learning applications for computer vision. The assignment will be delivered in the same groups from *Group assignment 1* and you start from this template notebook. The notebook you submit for grading is the last notebook you submit in the [Kaggle competition](https://www.kaggle.com/t/d11be6a431b84198bc85f54ae7e2563f) prior to the deadline on **Tuesday 24 May 23:59**. Closely follow [these instructions](https://github.com/gourie/kaggle_inclass) for joining the competition, sharing your notebook with the TAs and making a valid notebook submission to the competition. A notebook submission not only produces a *submission.csv* file that is used to calculate your competition score, it also runs the entire notebook and saves its output as if it were a report. This way it becomes an all-in-one-place document for the TAs to review. As such, please make sure that your final submission notebook is self-contained and fully documented (e.g. provide strong arguments for the design choices that you make). Most likely, this notebook format is not appropriate to run all your experiments at submission time (e.g. the training of CNNs is a memory hungry and time consuming process; due to limited Kaggle resources). It can be a good idea to distribute your code otherwise and only summarize your findings, together with your final predictions, in the submission notebook. For example, you can substitute experiments with some text and figures that you have produced "offline" (e.g. learning curves and results on your internal validation set or even the test set for different architectures, pre-processing pipelines, etc). We advise you to first go through the PDF of this assignment entirely before you really start. Then, it can be a good idea to go through this notebook and use it as your first notebook submission to the competition. You can make use of the *Group assignment 2* forum/discussion board on Toledo if you have any questions. Good luck and have fun!

---------------------------------------------------------------
NOTES:
* This notebook is just a template. Please keep the five main sections, but feel free to adjust further in any way you please!
* Clearly indicate the improvements that you make! You can for instance use subsections like: *3.1. Improvement: applying loss function f instead of g*.


# 1. Overview
This assignment consists of *three main parts* for which we expect you to provide code and extensive documentation in the notebook:
* Image classification (Sect. 2)
* Semantic segmentation (Sect. 3)
* Adversarial attacks (Sect. 4)

In the first part, you will train an end-to-end neural network for image classification. In the second part, you will do the same for semantic segmentation. For these two tasks we expect you to put a significant effort into optimizing performance and as such competing with fellow students via the Kaggle competition. In the third part, you will try to find and exploit the weaknesses of your classification and/or segmentation network. For the latter there is no competition format, but we do expect you to put significant effort in achieving good performance on the self-posed goal for that part. Finally, we ask you to reflect and produce an overall discussion with links to the lectures and "real world" computer vision (Sect. 5). It is important to note that only a small part of the grade will reflect the actual performance of your networks. However, we do expect all things to work! In general, we will evaluate the correctness of your approach and your understanding of what you have done that you demonstrate in the descriptions and discussions in the final notebook.

## 1.1 Deep learning resources
If you did not yet explore this in *Group assignment 1 (Sect. 2)*, we recommend using the TensorFlow and/or Keras library for building deep learning models. You can find a nice crash course [here](https://colab.research.google.com/drive/1UCJt8EYjlzCs1H1d1X0iDGYJsHKwu-NO).

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import random
import cv2

## 1.2 PASCAL VOC 2009
For this project you will be using the [PASCAL VOC 2009](http://host.robots.ox.ac.uk/pascal/VOC/voc2009/index.html) dataset. This dataset consists of colour images of various scenes with different object classes (e.g. animal: *bird, cat, ...*; vehicle: *aeroplane, bicycle, ...*), totalling 20 classes.

Our dataset is relatively small: 
The training set contains 749 examples and the test set contains 750 examples.

In [2]:
# Elina
#initstring = ../input/InputData/

# Nastya
#initstring = './kaggle/input/'

#Pili
initstring = '/kaggle/input/'

# Loading the training data
train_df = pd.read_csv(f'{initstring}kul-h02a5a-computer-vision-ga2-2022/train/train_set.csv', index_col="Id")
labels = train_df.columns
train_df["img"] = [np.load('{}kul-h02a5a-computer-vision-ga2-2022/train/img/train_{}.npy'.format(initstring, idx)) for idx, _ in train_df.iterrows()]
train_df["seg"] = [np.load('{}kul-h02a5a-computer-vision-ga2-2022/train/seg/train_{}.npy'.format(initstring, idx)) for idx, _ in train_df.iterrows()]
print("The training set contains {} examples.".format(len(train_df)))

# Show some examples
fig, axs = plt.subplots(2, 20, figsize=(10 * 20, 10 * 2))
for i, label in enumerate(labels):
    df = train_df.loc[train_df[label] == 1]
    axs[0, i].imshow(df.iloc[0]["img"], vmin=0, vmax=255)
    axs[0, i].set_title("\n".join(label for label in labels if df.iloc[0][label] == 1), fontsize=40)
    axs[0, i].axis("off")
    axs[1, i].imshow(df.iloc[0]["seg"], vmin=0, vmax=20)  # with the absolute color scale it will be clear that the arrays in the "seg" column are label maps (labels in [0, 20])
    axs[1, i].axis("off")
    
plt.show()

# The training dataframe contains for each image 20 columns with the ground truth classification labels and 20 column with the ground truth segmentation maps for each class
train_df.head(1)

In [3]:
# Loading the test data
test_df = pd.read_csv(f'{initstring}kul-h02a5a-computer-vision-ga2-2022/test/test_set.csv', index_col="Id")
test_df["img"] = [np.load('{}kul-h02a5a-computer-vision-ga2-2022/test/img/test_{}.npy'.format(initstring, idx)) for idx, _ in test_df.iterrows()]
test_df["seg"] = [-1 * np.ones(img.shape[:2], dtype=np.int8) for img in test_df["img"]]
print("The test set contains {} examples.".format(len(test_df)))

# The test dataframe is similar to the training dataframe, but here the values are -1 --> your task is to fill in these as good as possible in Sect. 2 and Sect. 3; in Sect. 6 this dataframe is automatically transformed in the submission CSV!
test_df.head(1)

## 1.3 Your Kaggle submission
Your filled test dataframe (during Sect. 2 and Sect. 3) must be converted to a submission.csv with two rows per example (one for classification and one for segmentation) and with only a single prediction column (the multi-class/label predictions running length encoded). You don't need to edit this section. Just make sure to call this function at the right position in this notebook.

In [4]:
def _rle_encode(img):
    """
    Kaggle requires RLE encoded predictions for computation of the Dice score (https://www.kaggle.com/lifa08/run-length-encode-and-decode)

    Parameters
    ----------
    img: np.ndarray - binary img array
    
    Returns
    -------
    rle: String - running length encoded version of img
    """
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    rle = ' '.join(str(x) for x in runs)
    return rle

def generate_submission(df):
    """
    Make sure to call this function once after you completed Sect. 2 and Sect. 3! It transforms and writes your test dataframe into a submission.csv file.
    
    Parameters
    ----------
    df: pd.DataFrame - filled dataframe that needs to be converted
    
    Returns
    -------
    submission_df: pd.DataFrame - df in submission format.
    """
    df_dict = {"Id": [], "Predicted": []}
    for idx, _ in df.iterrows():
        df_dict["Id"].append(f"{idx}_classification")
        df_dict["Predicted"].append(_rle_encode(np.array(df.loc[idx, labels])))
        df_dict["Id"].append(f"{idx}_segmentation")
        df_dict["Predicted"].append(_rle_encode(np.array([df.loc[idx, "seg"] == j + 1 for j in range(len(labels))])))
    
    submission_df = pd.DataFrame(data=df_dict, dtype=str).set_index("Id")
    submission_df.to_csv("submission.csv")
    return submission_df

# 2. Preprocessing the data

#### Balance

We took a look at the number of object in each class and noticed that the category person has significantly more objects than the other class. This might give problems in the future classification since introduces a bias caused by class imbalance. Since our dataset is relatively small (training set contains 749 examples and the test set 750 examples), we decided to augment all the classes except person. This was done by flipping both the segmentation and original images and changing the brightness of the original image.

In [5]:
def augmentData(data):
    labels = data.columns
    pp = labels.get_loc("person") ##Position person
    for i in range(data["img"].shape[0]):
        select= data.loc[i]
        if select[labels[pp]]==0:
        
            FLR = select.copy()
            FLR["img"] = np.array(tf.image.flip_left_right(FLR["img"]))
            FLR["seg"] = np.array(tf.image.flip_left_right(tf.expand_dims(FLR["seg"],axis=2)))
            data = data.append(FLR,ignore_index=True)
        
#             FUD = select.copy()
#             FUD["img"] = np.array(tf.image.flip_up_down(FUD["img"]))
#             FUD["seg"] = np.array(tf.image.flip_up_down(tf.expand_dims(FUD["seg"],axis=2)))
#             data = data.append(FUD,ignore_index=True)
        
            B = select.copy()
            B["img"] = np.array(tf.image.random_brightness(B["img"],0.6, seed=None))
            data = data.append(B,ignore_index=True)
            
            B = select.copy()
            B["img"] = np.array(tf.image.random_brightness(B["img"],0.4, seed=None))
            data = data.append(B,ignore_index=True)
            
    return data

#### Resize
The images did not have the same size at the beginning, so we searched for the biggest proportions (500x500) and resized the images to that particular size. Later on, we had memory problems with big RGB images, so we were forced to make these images smaller and resize them to 128x128.

#### Normalise & Filtering
A normalisation was also performed on the resized images, by dividing the array values by 255 as well as a gaussian blur filter. When these preprocessing steps were completed, we converted both the image en segmentation image to tensors, to make it possible to feed the system with it.

#### Grayscale
A discussion appeared when we wanted to feed the images to the neural network. Depending on the object of classification, grayscale or RGB should give better results. Since grayscale images are less costly than RGB, we first tried to augment grayscale images and use them to train on. We quickly understood that this results were less good than if we used augmented RGB images. This by the reason that our problem tries to classify objects, and their colour plays an important role as well in learning a model.

In [6]:
class Preprocessor:
    
    def __init__(self, img_size, test):
        self.img_size = img_size
        self.width = self.img_size[1]
        self.height = self.img_size[0]
        self.test = test
        
    def resize(self,img):
        resized = cv2.resize(img, (self.width,self.height), interpolation = cv2.INTER_AREA)
        return resized
    
    def normalization(self,gray_img):
        img = gray_img / 255.0
        return img
    
    def Filter(self,img,filter):
        if filter=='Gaussian':
            img = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
        elif filter=='Median':
            img = cv2.medianBlur(img,5)
        else:
            raise Error
            
        return img
    
    def preprocess(self,data):
        preprocData = data.copy()
        images = data["img"]
        segmentations = data["seg"]
        nb_images = images.shape[0]
        for i in range(nb_images):
            image = images[i]
            seg = segmentations[i]
            # Resize both img and seg
            image = self.resize(image)
            if self.test == False:
                seg = self.resize(seg)
            # Normalization
            normalized_img = self.normalization(image)
            # Filter 
            blur_img = self.Filter(normalized_img,'Gaussian')
            img = tf.convert_to_tensor(blur_img)
            seg = tf.convert_to_tensor(seg)
            images[i]=img
            segmentations[i]=seg
            
        preprocData['img'] = images
        preprocData['seg'] = segmentations
        return preprocData
    
    def __call__(self, data):
        return self.preprocess(data)
        

def FindLargestDim(data):
    imgs = data['img']
    x_max = 0
    y_max = 0
    for img in imgs:
        x,y,_ = img.shape
        if x>x_max:
            x_max=x
        if y>y_max:
            y_max=y
    return (x_max,y_max)

In [7]:
# Balance data set
# Choose if you want to use images with 3 dimensions (in RGB) or the grayscale images
# For transfer learning we need images in RGB values
preprocessed_train_df = train_df.copy()
test = False

preprocessed_train_df = augmentData(preprocessed_train_df)
    
# dim = FindLargestDim(preprocessed_train_df)
dim = (128,128)
preprocessor = Preprocessor(dim, test)
preprocessed_train_df = preprocessor(preprocessed_train_df)

In [8]:
print(len(train_df['img']))
print(len(preprocessed_train_df['img']))
print(preprocessed_train_df['img'][600].shape)
plt.imshow(preprocessed_train_df['img'][600])

In [9]:
# checking how balanced the dataset becomes after preprocessing
for i in range(len(preprocessed_train_df.columns)-2):
    arr = preprocessed_train_df[preprocessed_train_df.columns[i]].to_numpy()
    amount = np.count_nonzero(arr == 1)

    print('Number of', preprocessed_train_df.columns[i], ':', amount)

Now the images need to be put in datastructures which are compatible with the CNN's used

output is a so called one-hot encoding. One-hot encoding can be used when as loss the categorical-entropy is used.

In [10]:
SIZE_IMG = dim[0]

y = preprocessed_train_df[['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']]
train_yC = y.to_numpy()
print(train_yC)

# reshape images 
# input images should have a shape [batch_size, img_height, img_width, number_of_channels]
numberOfIm = len(preprocessed_train_df["img"])
dimIm = 3
    
train_x = np.array(np.zeros([numberOfIm, SIZE_IMG, SIZE_IMG, dimIm]))
for i in range(numberOfIm):
    Im = preprocessed_train_df["img"][i]
    A = np.asarray(Im).reshape(SIZE_IMG,SIZE_IMG,dimIm)
    train_x[i,:,:,:] = A

Same preprocess procedure for the test data

In [11]:
# same preprocessing procedure for the test data
test = True

preprocessed_test_df = test_df.copy()
# dim = FindLargestDim(test_df)
dim = (128,128)
preprocessor = Preprocessor(dim, test)
preprocessed_test_df = preprocessor(preprocessed_test_df)

In [12]:
print(preprocessed_test_df['img'][2].shape)
print(len(preprocessed_test_df['img']))
plt.imshow(preprocessed_test_df['img'][2])

In [13]:
# reshape images 
# input images should have a shape [batch_size, img_height, img_width, number_of_channels]
numberOfIm = len(preprocessed_test_df["img"])
test = np.array(np.zeros([numberOfIm, SIZE_IMG, SIZE_IMG, dimIm]))
for i in range(numberOfIm):
    Im = preprocessed_test_df["img"][i]
    A = np.asarray(Im).reshape(SIZE_IMG,SIZE_IMG,dimIm)
    test[i,:,:,:] = A

In [14]:
print(test.shape)

# 3. Image classification
The goal here is simple: implement a classification CNN and train it to recognise all 20 classes (and/or background) using the training set and compete on the test set (by filling in the classification columns in the test dataframe). We designed two main models, one model was build from scratch, and the other one uses transfer learning to make predictions.

## 3.1 Image classification from scratch

We tried several models and finally converged to the blog proposed by a following blog: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html . This model was finetuned and the hyperparameters were finetuned as much as possible. But still, after a lot of effort, the model could not find a relatively good solution. So we have decided to put more effort in the transfer learning model.

### Journey

After several experiments, we converged that th adam optimizer was the best solution for our problem. The trick was to set the learning rate small enough to let it converge without using too much epochs.
It is important to notice that the input are the resized images in RGB, and the output layer contains 20 classes. Most sources recommended to use a sigmoid function at the end and a binary cross-entropy loss function.

### Model 1

The goal here is simple: implement a classification CNN and train it to recognise all 20 classes (and/or background) using the training set and compete on the test set (by filling in the classification columns in the test dataframe).

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [62]:
# import necessary packages
import matplotlib.image as mpimg
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [16]:
# # model that was proposed in the blog
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32,(3,3),activation = "relu" , input_shape = (128,128,3)) ,
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
#     tf.keras.layers.MaxPooling2D(2,2),
#     # added
#     tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
#     tf.keras.layers.AveragePooling2D(2,2),
    
#     tf.keras.layers.Flatten(), 
#     tf.keras.layers.Dense(64,activation="relu"),      #Adding the Hidden layer
#     tf.keras.layers.Dropout(0.2, seed = 2019),
#     tf.keras.layers.Dense(20,activation = "sigmoid")   #Adding the Output Layer
# ])

In [17]:
# # Model 2
# ## This is the first model we tried, the predictions are not that good. After 1 epoch overfit
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (128,128,3)) ,
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Flatten(), 
#     tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
#     tf.keras.layers.Dropout(0.1,seed = 2019),
#     #tf.keras.layers.Dense(400,activation ="relu"),
#     #tf.keras.layers.Dropout(0.3,seed = 2019),
#     #tf.keras.layers.Dense(300,activation="relu"),
#     #tf.keras.layers.Dropout(0.4,seed = 2019),
#     #tf.keras.layers.Dense(200,activation ="relu"),
#     #tf.keras.layers.Dropout(0.2,seed = 2019),
#     tf.keras.layers.Dense(20,activation = "sigmoid")   #Adding the Output Layer: kleine getalletjes want absolute kansen (softmaxgeeft relatieve kansen, dus is niet dat hij heel onaccuraat is)
# ])

In [18]:
# model.summary()

[COPYRIGHT] So far, we've seen RMSProp and Momentum take contrasting approaches. While momentum accelerates our search in direction of minima, RMSProp impedes our search in direction of oscillations. Adam or Adaptive Moment Optimization algorithms combines the heuristics of both Momentum and RMSProp.


In [19]:
# # binary cross entropy: multiple labels per photo 1 000.  11000
# from tensorflow.keras.optimizers import RMSprop,SGD,Adam

# # TODO: overfit wrsch: binary of cross: want hoe bepaal je accuracty en loss nu? want multilabel nu, 
# # maar kijkt hij naar 1 of 2 values nu?
# # probleem: val los wordt hoger ipv lager$

# # adam: lr groot -> verklein. 0.1 -> 0.001 -> 0.000001
# # wat is rmsprop -> opzoeken (adam goed)
# adam = Adam(lr=0.01)

# model.compile(#optimizer='rmsprop',
#               optimizer='adam', 
#               loss='binary_crossentropy', 
#               metrics=['accuracy'])

In [20]:
# # This callback will stop the training when there is no improvement in
# # the loss for three consecutive epochs.
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# checkpoint_filepath = 'checkpoint.h5'
# # Model weights are saved at the end of every epoch, if it's the best seen so far.
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=True,
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=True)


# history = model.fit(train_x, train_yC, validation_split=0.2, epochs=500, batch_size=32, verbose=1, callbacks=[callback, model_checkpoint_callback])

# # The model weights (that are considered the best) are loaded into the model.
# model.load_weights(checkpoint_filepath)

# model.save('model_1.h5')

In [21]:
# # plotting the accuracy and loss functions
# # list all data in history
# print(history.history.keys())
# # summarize history for accuracy
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])   # Terug toeveogen wanneer validatieset hebben
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [22]:
#import keras
#model = keras.models.load_model('ModelFromScracth.h5')

In [23]:
# # make a prediction on the test set
# yhat = model.predict(test)
# print(yhat[0])

In [24]:
# # choose threshold to convert to zero's and ones
# yhat = yhat > 0.1
# yhat = yhat.astype(int)

In [25]:
# import sys
# import numpy
# numpy.set_printoptions(threshold=sys.maxsize)

# # yhat

In [26]:
# ## Testing with 1 image
# plt.imshow(test_df["img"][1])

In [27]:
# testImage = test[1,:,:,:]
# pred = model.predict(testImage.reshape(1,128,128,3))

In [28]:
# print(pred)
# print(np.argmax(pred))
# top = np.argsort(pred[0])[:-4:-1]
# print(top)
# for i in range(len(top)):
#     index = top[i]
#     print(train_df.columns[index])

In [29]:
# plt.imshow(test_df["img"][2])

In [30]:
# testImage2 = test[2,:,:,:]
# pred = model.predict(testImage2.reshape(1,128,128,3))

In [31]:
# print(pred)
# print(np.argmax(pred))
# top = np.argsort(pred)[:-4:-1]
# print(top)
# for i in range(len(top)):
#     index = top[i]
#     print(train_df.columns[index])

In [32]:
# plt.imshow(test_df["img"][3])

In [33]:
# testImage3 = test[3,:,:,:]
# pred = model.predict(testImage2.reshape(1,128,128,3))

In [34]:
# print(pred)
# print(np.argmax(pred))
# top = np.argsort(pred[0])[:-4:-1]
# print(top)
# for i in range(len(top)):
#     index = top[i]
#     print(train_df.columns[index])

## 3.2 Image classification with transfer learning

For transfer learning we tried a lot of different models and added a lot of different layers. The model which performed the best is given below. 

In [35]:
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model, load_model

As base model we tried ResNet50, ResNet101, VGG16, VGG19, InceptionV3 and DenseNet. Since DenseNet gave the best results, we chose this as the base model. To the base model we added following layers:
* Flatten: This layer makes from a matrix a array. This ensure the input has the right shape.
* Batchnormalization: This layer applies a transformation that maintains the mean output close to 0 and the output standard deviation close to 1. The purpose of this is to avoid exploding or fanishing gradients, which happens when the gradient becomes too large or too small.
* Dense: A normal layer which calculates the activation function of the input multiplied with the weight of the corresponding neuron. 
* Dropout: The Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time, which helps prevent overfitting. Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all inputs is unchanged.
* Dense: again a normal layer to calculate the activation function, however this time the activation function is set to 'sigmoid', a non-linear function. This is a good last activation function for the type of output used in this assignment.

After constructing the model, the model needs to be compiled. The Adam function to minimize the loss gave the best results. 

After compiling the function the training images and corresponding labels are put togheter in one dataset. This approach proved the give better results than when we keep them apart. Here the trainig set is also split in a training and validation set. The validation set is needed to prevent overfitting. Overfitting happens when a model follows exactly the training samples and doesn't generalise well. Since the model is not trained on the validation set, the loss of the model on the validation set will get worse when the model is overfitted. Therefore we know the training is converged as soon as the loss of the validation set starts to increase instead of decrease. 

In [36]:
from keras import backend as K

K.set_learning_phase(0)

numClasses = 20

# base_model = ResNet101(weights="imagenet", include_top=False, pooling='max', input_shape=(128, 128, 3))
# for layer in base_model.layers:
#     layer.trainable = False
base_model=tf.keras.applications.DenseNet121(input_shape=[128,128,3], 
                                       include_top=False, 
                                       weights='imagenet') 
    
K.set_learning_phase(1)

top_model = Sequential()
#vgg16 = tf.keras.applications.vgg16
#transferModel.add(vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(500, 500, 3))) -> kernel dies
#transferModel.add(ResNet50(weights="imagenet", include_top=False, pooling='max', input_shape=(500, 500, 3)))
top_model.add(Flatten())
top_model.add(BatchNormalization(renorm=True))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dropout(0.1))
# top_model.add(Dense(216, activation='relu'))
top_model.add(Dense(numClasses, activation='sigmoid'))
#transferModel.layers[0].trainable = False

transferModel = Model(base_model.input, top_model(base_model.output))

In [37]:
# transferModel.summary()

In [38]:
adam=tf.keras.optimizers.Adam(lr=1e-6)
transferModel.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
from sklearn.model_selection import train_test_split
trainsplit_XC, val_XC,trainsplit_yC, val_yC = train_test_split(train_x,train_yC, 
                                                   test_size=0.2, 
                                                   random_state=0
                                                  )
train_dataC = tf.data.Dataset.from_tensor_slices((trainsplit_XC, trainsplit_yC))
test_dataC = tf.data.Dataset.from_tensor_slices((val_XC, val_yC))
BATCH_SIZE = 12
BUFFER_SIZE = 1000
train_batchesC = train_dataC.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_batchesC = train_batchesC.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_batchesC = test_dataC.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
test_batchesC = test_batchesC.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [40]:
history = transferModel.fit(train_batchesC,validation_data=test_batchesC,steps_per_epoch=trainsplit_XC.shape[0]//BATCH_SIZE, epochs=50,validation_steps=val_XC.shape[0]//BATCH_SIZE, batch_size=BATCH_SIZE, shuffle=True)

The code below shows plots of the accuracy and the loss. As can be seen on the plots the validation loss starts increasing again after 40 epochs. This means the model will start to overfit and we should stop the training process.

In [ ]:
# plotting the accuracy and loss functions
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])   # Terug toeveogen wanneer validatieset hebben
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Now, it's time to look at some results. An image of the test set is shown below. There is a sofa on the image visible. The ouput of the prediction is the chance the image contains the object represented by the label. The three labels which had the biggest chance of being in the image contains also a sofa. Although this didn't get the highest chance, the model still got close. 

In [ ]:
plt.imshow(test_df["img"][2])

In [ ]:
testImage = test[2,:,:,:]
pred = transferModel.predict(testImage.reshape(1,128,128,3))
top = np.argsort(pred[0])[:-4:-1]
for i in range(len(top)):
    index = top[i]
    print(train_df.columns[index])

Let's try another image. Image 63 contains a cat as can be seen below. The model did a really good prediction for this one. As the highest chance of the image containing a label was given to the label of the cat. 

In [ ]:
plt.imshow(test_df["img"][63])

In [ ]:
testImage = test[63,:,:,:]
pred = transferModel.predict(testImage.reshape(1,128,128,3))
top = np.argsort(pred[0])[:-4:-1]
for i in range(len(top)):
    index = top[i]
    print(train_df.columns[index])

A last example is a image with a dog on the beach. The model did not perform well for this image, since it predicts the image contains a aeroplane, bird and a boat. The image could be difficult to classify because the dog is not that different from the water. The colors look a bit the same and the water is not smooth behind the dog. 

In [ ]:
plt.imshow(test_df["img"][28])

In [ ]:
testImage = test[28,:,:,:]
pred = transferModel.predict(testImage.reshape(1,128,128,3))
top = np.argsort(pred[0])[:-4:-1]
for i in range(len(top)):
    index = top[i]
    print(train_df.columns[index])

Below, the output of the model is processed so it can put in the csv file. The output of the model are all chances, so the ouput needs to be converted to zeros and ones. As some images contain multiple labels the three highest labels are set to one, the other labels are set to zero.

In [ ]:
yhat = transferModel.predict(test)

In [ ]:
yhat

In [ ]:
# yhat = yhat > 0.99999
# yhat = yhat.astype(int)

In [ ]:
print(labels)
yhat[7]

In [ ]:
plt.imshow(test_df["img"][7])

In [ ]:
for j in range(yhat.shape[0]):
# for j in range(2):
    top = np.argsort(yhat[j])[:-4:-1]
    for i in range(len(top)):
        index = top[i]
        yhat[j,index]=1
        
yhat = yhat == 1
yhat = yhat.astype(int)

# 4. Semantic segmentation
The goal here is to implement a segmentation CNN that labels every pixel in the image as belonging to one of the 20 classes (and/or background). Use the training set to train your CNN and compete on the test set (by filling in the segmentation column in the test dataframe).

In [ ]:
class RandomSegmentationModel:
    """
    Random segmentation model: 
        - generates random label maps for the inputs based on the class distributions observed during training
        - every pixel in an input can only have one label
    """
    def fit(self, X, Y):
        """
        Adjusts the class ratio variable to the one observed in Y. 

        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
        Y: list of arrays - n x (height x width)

        Returns
        -------
        self
        """
        self.distribution = np.mean([[np.sum(Y_ == i) / Y_.size for i in range(len(labels) + 1)] for Y_ in Y], axis=0)
        print("Setting class distribution to:\nbackground: {}\n{}".format(self.distribution[0], "\n".join(f"{label}: {p}" for label, p in zip(labels, self.distribution[1:]))))
        return self
        
    def predict(self, X):
        """
        Predicts for each input a label map.
        
        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
            
        Returns
        -------
        Y_pred: list of arrays - n x (height x width)
        """
        np.random.seed(0)
        return [np.random.choice(np.arange(len(labels) + 1), size=X_.shape[:2], p=self.distribution) for X_ in X]
    
    def __call__(self, X):
        return self.predict(X)
    
model = RandomSegmentationModel()
model.fit(train_df["img"], train_df["seg"])
test_df.loc[:, "seg"] = model.predict(test_df["img"])
test_df.head(1)

## 4.1 Training-data for semantic segmentation
For semantic segmentation the training-data for the model is a tensorflow dataset. The trainig data is split into a trainig set and a validation set. Further, the images and the segmentation images are set into a dataset.

In [ ]:
dimImy = 1
train_yS = np.array(np.zeros([preprocessed_train_df["seg"].shape[0], dim[0], dim[1], dimImy]))
for i in range(preprocessed_train_df["seg"].shape[0]):
    Im = preprocessed_train_df["seg"][i]
    A = np.asarray(Im).reshape(dim[0],dim[1],dimImy)
    train_yS[i,:,:,:] = A

In [ ]:
from sklearn.model_selection import train_test_split
trainsplit_X, val_X,trainsplit_y, val_y = train_test_split(train_x,train_yS, 
                                                   test_size=0.2, 
                                                   random_state=0
                                                  )

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((trainsplit_X, trainsplit_y))
val_data = tf.data.Dataset.from_tensor_slices((val_X, val_y))
BATCH_SIZE = 12
BUFFER_SIZE = 1000
train_batches = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_batches = train_batches.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_batches = val_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
val_batches = val_batches.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


## 4.2 Model from scratch
The model made from scratch is U-Net. This type of framework has showcased good ability for sematic segmentation. We used pixel based method and not contour based method.
For the U-net model we resized all the images to 128x128x3 size and gave them as input to the model. Our model is made up of down-block, up-block and bottleneck.
The down-block is used for : downsampling of the network. we used a kernel of 3x3 and a dropout of 0.05. For a higher value of dropout we had much worse results.
The up-block is used for: we used this to have bigger layers in our CNN, so as to up size it. Again we keep then dropout layer to 0.05.

In our model we have used 11 convoltion layers where we increase the size of neurons from 16, 32, 64, 128, 256 and bottelneck at 512. Then we go down again to 256 128 64 32 and 16.
The final output is a tensor of 128x128x21 with 21 being the classes including the background. This output contains probabilities of every pixel belonging to that particular class.
We take the class value for every pixel which has the highest probablility.

The optimizer we used was Adam. The parameters for the models were choosen after perfroming several experiments. Our current model is able to detect the presence of an object but sometimes is not able to classify it correctly.


In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

In [ ]:
class Unet_model:
    def __init__(self, input_shape):
            self.input_shape=input_shape
            
    def down_block(self,x,size):
        conv = tf.keras.layers.Conv2D(size, (3, 3), kernel_initializer='he_normal',
                                    padding='same')(x)
        conv = tf.keras.layers.Activation('relu')(conv)
        conv = tf.keras.layers.Dropout(0.05)(conv)

        conv = tf.keras.layers.Conv2D(size, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                    padding='same')(conv)
        pool = tf.keras.layers.MaxPooling2D((2, 2))(conv)
        return pool,conv

    def bottleneck(self,x,size):
        conv = tf.keras.layers.Conv2D(size, (3, 3), kernel_initializer='he_normal',
                                    padding='same')(x)
        conv = tf.keras.layers.Activation('relu')(conv)
        conv = tf.keras.layers.Dropout(0.05)(conv)
        conv = tf.keras.layers.Conv2D(size, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                    padding='same')(conv)
        return conv

    def up_block(self,x,y,size):
    #     upconv = tf.keras.layers.Conv2DTranspose(size, (2, 2), strides=(2, 2), padding='same')(x)
        upconv = tf.keras.layers.UpSampling2D((2, 2))(x)
        upconv = tf.keras.layers.concatenate([upconv, y])
        conv = tf.keras.layers.Conv2D(size, (3, 3), kernel_initializer='he_normal',
                                    padding='same')(upconv)
        conv = tf.keras.layers.Activation('relu')(conv)
        conv = tf.keras.layers.Dropout(0.05)(conv)
        conv = tf.keras.layers.Conv2D(size, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                    padding='same')(conv)
        return conv

    def Model(self):
        """
        Function to build the UNet model that was used for the own model in image segemntation
        We experimented with both dropout and batchnormalization
        In some experiments the dropout and or batchnormalization was removed or tweaked
        """
        inputs = Input(self.input_shape)

        pool1,conv1 = self.down_block(inputs,16)
        pool2,conv2 = self.down_block(pool1,32)
        pool3,conv3 = self.down_block(pool2,64)
        pool4,conv4 = self.down_block(pool3,128)
        pool5,conv5 = self.down_block(pool4,256)

        conv6 = self.bottleneck(pool5,512)

        conv7 = self.up_block(conv6,conv5,256)
        conv8 = self.up_block(conv7,conv4,128)
        conv9 = self.up_block(conv8,conv3,64)
        conv10 = self.up_block(conv9,conv2,32)
        conv11 = self.up_block(conv10,conv1,16)

        outputs = tf.keras.layers.Conv2D(21, (1, 1), activation='sigmoid')(conv11)

        self.model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

        self.model.compile(optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
                name='Adam'),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['acc'])
#         self.model.summary()
        return self.model

    def fit(self,train_dataset,steps_train,Epochs,val_dataset=None,steps_val=None):
        self.model.fit_generator(train_dataset,validation_data=val_dataset,steps_per_epoch=steps_train,validation_steps = steps_val, epochs=Epochs)
        
    def predict(self,test_image):
        x,y,_ = test_image.shape
        test_image = np.array(test_image)
        test_im = cv2.resize(test_image, (128,128), interpolation = cv2.INTER_NEAREST)
        test_im = test_im / 255.0
        test_im = cv2.GaussianBlur(test_im,(5,5),cv2.BORDER_DEFAULT)
        trst_im = tf.convert_to_tensor(test_im)
        test_im = tf.expand_dims(test_im, axis=0)
        pr = self.model.predict(test_im)
        pred_mask = tf.argmax(pr[0], axis=-1)
        pred_mask = pred_mask[..., tf.newaxis]
        pred_mask = cv2.resize(np.array(pred_mask), (y,x), interpolation = cv2.INTER_NEAREST)
        return pred_mask
    
    def __call__(self):
        return self.Model()


In [ ]:
# input_shape = (dim[0], dim[1], 3)
# Unet =  Unet_model(input_shape)
# model_U = Unet()
# Unet.fit(train_batches,trainsplit_X.shape[0]//BATCH_SIZE, 100,val_batches,val_X.shape[0]//BATCH_SIZE)

In [ ]:
# P=Unet.predict(train_df["img"][0])
# fig, axs = plt.subplots(1,3)
# axs[0].imshow(train_df["img"][0])
# axs[1].imshow(train_df["seg"][0])
# axs[2].imshow(P)
# plt.show()

In [ ]:
# for i in range(test_df["img"].shape[0]):
#     P=Unet.predict(test_df["img"][i])
#     test_df["seg"][i]=P

In [ ]:
# print(test_df["seg"][6].shape)

In [ ]:
# n = 101
# plt.imshow(test_df["seg"][n])
# plt.show()
# plt.imshow(test_df["img"][n])
# plt.show()

## 4.3 Transfer Learning
The model from scratch isn't performing very well. We will see that a model based on a pretrained model will perform better. Different transfer learning model were implemented and tested.

In [ ]:
!pip install -q git+https://github.com/tensorflow/examples.git
from tensorflow_examples.models.pix2pix import pix2pix

For transfer learning we tested three different architectures to find the best one. We have compared MobilNetV2, DensNet and ResNet. MobilNetV2 was the first attempt to make the model work, it didn't perform very well and was not used further. In literature we found that ResNet perform well, it uses skip connections via addition. Finally, we tryed DensNet. DensNet outperformed the two previous tested architectures. DensNet uses dense connections between layers, that connects all layers with each other. This last architectures was fastest to train, it needed less epochs, and was more accurate than the two others. The same optimizer and loss is used for semantic segmentation with transfer learning, namely Adam optimizer and categorical crossentropy as loss function. Adam optimizer is a gradient descent method, this is an iterative method and perform very well, it is used in a lot of applications. Science we have a multi segmentation task to do, categorical crossentropy as loss function is an obvious choice. It allows using more than two labels.

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_resnet50_unet(input_shape,output_channels):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained ResNet50 Model """
    resnet50 = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)

    """ Encoder """
    s1 = resnet50.get_layer(resnet50.layers[0].name).output           ## (512 x 512)
    s2 = resnet50.get_layer("conv1_relu").output        ## (256 x 256)
    s3 = resnet50.get_layer("conv2_block3_out").output  ## (128 x 128)
    s4 = resnet50.get_layer("conv3_block4_out").output  ## (64 x 64)

    """ Bridge """
    b1 = resnet50.get_layer("conv4_block6_out").output  ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)

    """ Output """
    outputs = Conv2D(output_channels, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="ResNet50_U-Net")
    return model

In [ ]:
class SemanticSegmentationTL:
    
    def __init__(self,model,input_shape,nb_channels):
        self.model = model
        self.input_shape = input_shape
        self.nb_channels = nb_channels
        
    def unet_model(self):
        inputs = tf.keras.layers.Input(shape=self.input_shape)
        # Downsampling through the model
        skips = self.down_stack(inputs)
        x = skips[-1]
        skips = reversed(skips[:-1])
        # Upsampling and establishing the skip connections
        for up, skip in zip(self.up_stack, skips):
            x = up(x)
            concat = tf.keras.layers.Concatenate()
            x = concat([x, skip])
        # This is the last layer of the model
        last = tf.keras.layers.Conv2DTranspose(
        filters=self.nb_channels, kernel_size=3, strides=2,
            padding='same')  #64x64 -> 128x128

        x = last(x)
        return tf.keras.Model(inputs=inputs, outputs=x)

        
    def Model(self):
        if self.model=="MobileNetV2":
            base_model = tf.keras.applications.MobileNetV2(input_shape=self.input_shape, include_top=False)
            # Use the activations of these layers
            layer_names = [
                'block_1_expand_relu',   # 64x64
                'block_3_expand_relu',   # 32x32
                'block_6_expand_relu',   # 16x16
                'block_13_expand_relu',  # 8x8
                'block_16_project',      # 4x4
            ]
            base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

            # Create the feature extraction model
            self.down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

            self.down_stack.trainable = False
            self.up_stack = [
                pix2pix.upsample(512, 3),  # 4x4 -> 8x8
                pix2pix.upsample(256, 3),  # 8x8 -> 16x16
                pix2pix.upsample(128, 3),  # 16x16 -> 32x32
                pix2pix.upsample(64, 3),   # 32x32 -> 64x64
            ]
            self.modelTL = self.unet_model()
            
        elif self.model=="Densnet":
            base_model = tf.keras.applications.DenseNet121(input_shape=self.input_shape, 
                                       include_top=False, weights='imagenet') 
            layer_names = ['conv1/relu', # size 64*64
                          'pool2_relu',  # size 32*32
                          'pool3_relu',  # size 16*16
                          'pool4_relu',  # size 8*8
                          'relu'        # size 4*4
                          ] 
            base_model_outputs = [base_model.get_layer(name).output for name in layer_names]
            self.down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

            self.down_stack.trainable = False
            self.up_stack = [
                pix2pix.upsample(512, 3),  # 4x4 -> 8x8
                pix2pix.upsample(256, 3),  # 8x8 -> 16x16
                pix2pix.upsample(128, 3),  # 16x16 -> 32x32
                pix2pix.upsample(64, 3),   # 32x32 -> 64x64
            ]
            self.modelTL = self.unet_model()
        elif self.model=="Resnet":
            self.modelTL = build_resnet50_unet(self.input_shape,self.nb_channels)
            
        self.modelTL.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
                name='Adam'),
             metrics=['accuracy']) 
        return self.modelTL
            
    def fit(self,train_dataset,steps_train,Epochs,val_dataset=None,steps_val=None):
        hist = self.modelTL.fit_generator(train_dataset, epochs=Epochs,validation_data=val_dataset,steps_per_epoch=steps_train,validation_steps = steps_val)
        return hist
    
    def  predict(self,test_image):
        x,y,_ = test_image.shape
        test_image = np.array(test_image)
        test_im = cv2.resize(test_image, (128,128), interpolation = cv2.INTER_NEAREST)
        test_im = test_im / 255.0
        test_im = cv2.GaussianBlur(test_im,(5,5),cv2.BORDER_DEFAULT)
        trst_im = tf.convert_to_tensor(test_im)
        test_im = tf.expand_dims(test_im, axis=0)
        pr = self.modelTL.predict(test_im)
        pred_mask = tf.argmax(pr[0], axis=-1)
        pred_mask = pred_mask[..., tf.newaxis]
        pred_mask = cv2.resize(np.array(pred_mask), (y,x), interpolation = cv2.INTER_NEAREST)
        return pred_mask
            
    def __call__(self):
        return self.Model()

In [ ]:
input_shape = (dim[0], dim[1], 3)
TLSS =  SemanticSegmentationTL("Densnet",input_shape,21)

MODELSS = TLSS()
MODELSS.summary()

In [ ]:
hist = TLSS.fit(train_batches,trainsplit_X.shape[0]//BATCH_SIZE, 30,val_batches,val_X.shape[0]//BATCH_SIZE)

In [ ]:
loss = hist.history['loss']
val_loss = hist.history['val_loss']

acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']


fig, axs = plt.subplots(1,2)
axs[0].plot(hist.epoch, loss, 'r', label='Training loss')
axs[0].plot(hist.epoch, val_loss, 'bo', label='Validation loss')
axs[0].set_title('Training and Validation Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss Value')
axs[0].set_ylim([0, 1])
axs[0].legend()
axs[1].plot(hist.epoch, acc, 'r', label='Training accuracy')
axs[1].plot(hist.epoch, val_acc, 'bo', label='Validation accuracy')
axs[1].set_title('Training and Validation Accuracy')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
axs[1].set_ylim([0, 1])
axs[1].legend()

The model is trained and now we can predict the segmentations for the test data set.

In [ ]:
for i in range(test_df["img"].shape[0]):
    P=TLSS.predict(test_df["img"][i])
    test_df["seg"][i]=P

The prediction for the training data looks very similar to the real segmentation image. The example shown here, can easily reconstruct the bird in the predicted segmentation image.

In [ ]:
n=460
P=TLSS.predict(train_df["img"][n])
fig, axs = plt.subplots(1,3)
axs[0].imshow(train_df["img"][n])
axs[1].imshow(train_df["seg"][n])
axs[2].imshow(P)
plt.show()
print('nb of pixels labeld as bird in real segmentation: '+str(train_df["seg"][n][train_df["seg"][n]==labels.get_loc("bird")+1].shape))
print('nb of pixels labeld as bird in prediction: '+str(P[P==labels.get_loc("bird")+1].shape))

The prediction from the validation set is very different, some images can be segmented easily, others have difficulties e.g. noisy images are segmented difficultly. 

In [ ]:
for image, mask in val_batches.take(1):
    pred_mask = MODELSS.predict(image)
    pred_mask = tf.argmax(pred_mask[0], axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    fig, axs = plt.subplots(1,3)
    axs[0].imshow(image[0])
    axs[1].imshow(mask[0])
    axs[2].imshow(pred_mask)
    plt.show()

Om the image below we see a bird. This image is segmented pretty well. Images with few different object is segmented accuratly enough.

In [ ]:
n=22
P=TLSS.predict(test_df["img"][n])
fig, axs = plt.subplots(1,2)
axs[0].imshow(test_df["img"][n])
axs[1].imshow(test_df["seg"][n])
plt.show()
print('nb of pixels labeld as bird: '+str(test_df["seg"][n][test_df["seg"][n]==labels.get_loc("bird")+1].shape))

Compared to the previous image, this image contains a lot of different objects. In the segmentation image we see different objects in different location, it recongnize the class of the object well, but has problems to find the full/correct location of it.

In [ ]:
n=62
P=TLSS.predict(test_df["img"][n])
fig, axs = plt.subplots(1,2)
axs[0].imshow(test_df["img"][n])
axs[1].imshow(test_df["seg"][n])
plt.show()
print('nb of pixels labeld as tvmonitor: '+str(test_df["seg"][n][test_df["seg"][n]==labels.get_loc("tvmonitor")+1].shape))
print('nb of pixels labeld as person: '+ str(test_df["seg"][n][test_df["seg"][n]==labels.get_loc("person")+1].shape))

For this model only 30 epochs are used, increasing this value should ensure a beter performance. Having more time, we could further experiment with other architectures and more paramters.

## Submit to competition
You don't need to edit this section. Just use it at the right position in the notebook. See the definition of this function in Sect. 1.3 for more details.

In [ ]:
# export solutions to good format
test_df.loc[:, labels] = yhat
test_df.head(1)

In [ ]:
generate_submission(test_df)


# 5. Adversarial attack
For this part, we try to fool the classification and CNN, using an *adversarial attack*. More specifically, the goal is build a CNN to perturb test images in a way that (i) they look unperturbed to humans; but (ii) the CNN classifies these images in line with the perturbations.

### Fast Gradient Sign Method

The fast gradient sign method is a kind of adversial attack based on the white-box approach. It is called like this because it uses the sign of the gradient to add noice in the same direction as the gradient to the image. The noice is scaled by epsilon. 

In [46]:
from tensorflow.keras.losses import categorical_crossentropy

The function below, constructs the gradient signs for the image

In [47]:
def generate_image_adversary(model, image, label):
    # cast the image
    image = tf.cast(image, tf.float32)
    # record our gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our image should be tacked for gradient updates
        tape.watch(image)
        # use our model to make predictions on the input image and then compute the loss   
        pred = model(tf.reshape(image, (1,128,128,3)))
        loss = categorical_crossentropy(label, pred)
    # calculate the gradients of loss with respect to the image, then compute the sign of the gradient
    gradient = tape.gradient(loss, image)
    signedGrad = tf.sign(gradient)
    # construct the image adversary
    return signedGrad

Below an example for the attack is given. First an image is shown and then the CNN for the classification with transfer learning is used for predicting the labels for this image. As can be seen, the CNN gives a good result for this image.

In [51]:
plt.imshow(train_df["img"][4])

In [52]:
testImage = train_x[4,:,:,:]
pred = transferModel.predict(testImage.reshape(1,128,128,3))
top = np.argsort(pred[0])[:-4:-1]
for i in range(len(top)):
    index = top[i]
    print(train_df.columns[index])

Now an epsilon is chosen. The higher the epsilon, the bigger the noise is. When the epsilon is too high, it would be visible for the naked eye, however when choosing a moderate epsilon it is not visible. Then the signs of the gradient for this image combined with the CNN is calculated using the function above.

In [54]:
eps = 0.5
signedGrad = generate_image_adversary(transferModel, testImage, tf.constant(train_yC[4], shape=(1,20)))

Visualising the signs of the gradient reuslts in a black image. This is because the gradient results in all zeros. This means there will be no noise added to the image and the adversial attack will not work.

In [55]:
plt.imshow(signedGrad)

Visualizing the adversial image looks a bit different, however this is because this image is preprocessed. However it looks the same as the preprocessed image.

In [56]:
adversary = (testImage + (signedGrad * eps)).numpy()
plt.imshow(adversary)

In [57]:
predAdv = transferModel.predict(tf.reshape(adversary, (1,128,128,3)))

As said above, since the signedGrad contained only zeros, the image will not have changed and the CNN can predict the lables for this image. 

In [59]:
top = np.argsort(predAdv[0])[:-4:-1]
for i in range(len(top)):
    index = top[i]
    print(train_df.columns[index])

We are not sure why the gradient of the image only contains zeros. 

This kind of attack is a white box attack, this means the attacker must have full access to the model. Therefore in the real world, it is difficult to attack a system using this method. However, there also exist black-box attacks which can attack a model without knowlegde of the model. Therefore, the model needs some defense strategies to be more reliable for white box as well as black box attacks. A strategy to improve the robustness is the 'Adversial training'. It is a technique where the neural network is trained on adversial images. An adversial image can be constructed by using the fast gradient sign method as shown above. However, there are some other techniques. In short these techniques are: PGD adversial training, Ensemble adversarial training, Adversarial logit pairing and Generative adversarial training. Another strategy is 'Randomization'. A randomization-based defense attempts to randomize the adversarial effects into random effects, this is done by adding some extra layers in front of the neural network or in between. There are different methods which fall under this category, an example is 'Random input transformation'. In this technique the image, which is fed to the neural network, is first resized to a random size and then the image is randomly padded. Padding is adding zeros around the image. Then this image is fed to the neural network and the model can start to train. 

# 5. Discussion
Finally, take some time to reflect on what you have learned during this assignment. Reflect and produce an overall discussion with links to the lectures and "real world" computer vision.

The lectures gave a good introduction to different methods and gave some theoretical background of it. In the assignment some methods were tested on a application. On of the points that we saw is: despite of having a good theoretical model it will not work if it hasn't had a good preprocessing on the input.

The classification and segmentation models worked decently for images that were very clear with the object and the boundaries of the object. With transfer learning model we have good results for both.
When there is an overlap the output is not great. For some images where classification works correctly we have proper implementation of the adversary attack.

In data augmentation we must note that rotating the images worsens the segmentation and classification resukts but flipping it is good enough.

Some more time could be spent in modifying the model for segmentation and classification for the ones we have to make from scratch.
Our models can only be applied to cases where the images are very clear with proper distinction of background and object.
